In [19]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import copy

import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

from keras.models import Sequential
from keras.layers import Dense

In [21]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [15]:
class Graph(object):
    def __init__(self, m, n):
        self.m = m
        self.n = n
        self.VerticesMatrix = np.zeros((self.m, self.n))
        self.__fill_matrix()
        self.edges = set()
        self.DictEdges = dict()
        self.DictEdgesInv = dict()
        self.DictResistance = dict()
        self.__get_edges()
        self.n_connections = self.get_number_connections()
        self.pairs_for_break = self.pairs_for_break()

    def __fill_matrix(self):
        k = 0
        for i in range(self.m):
            for j in range(self.n):
                self.VerticesMatrix[i][j] = k
                k += 1

    def __get_edges(self):
        def add_pair(pair):
            pair = [int(i) for i in pair]
            pair = tuple(sorted(pair))
            if pair not in self.edges:
                self.edges.add(pair)
                self.DictEdges[len(self.DictEdges)+1] = list(pair)
                self.DictEdgesInv[pair] = len(self.DictEdgesInv)+1
                # self.DictResistance[len(self.DictResistance)+1] = np.random.randint(10, 50)
                self.DictResistance[len(self.DictResistance)+1] = 10

        for i in range(self.m):
            for j in range(self.n):
                if i != self.m-1: add_pair([self.VerticesMatrix[i][j], self.VerticesMatrix[i+1][j]])
                if i != 0: add_pair([self.VerticesMatrix[i][j], self.VerticesMatrix[i-1][j]])
                if j != 0: add_pair([self.VerticesMatrix[i][j], self.VerticesMatrix[i][j-1]])
                if j != self.n-1: add_pair([self.VerticesMatrix[i][j], self.VerticesMatrix[i][j+1]])

    def get_connections(self):
        nums = [*self.VerticesMatrix[0, :], *self.VerticesMatrix[1:-1, -1],
                *self.VerticesMatrix[-1, :], *self.VerticesMatrix[1:-1, 0]]
        nums = [int(i) for i in nums]
        return list(it.combinations(nums, 2))
    
    def get_number_connections(self):
        return len(self.get_connections())

    def get_incidence_matrix(self):
        def get_m(matrix):
            m = 0
            for i in range(matrix.shape[0]):
                for j in range(matrix.shape[1]):
                    if not np.isnan(matrix[i][j]): m += 1
            return m
        
        def get_vert_poz(matrix, v):
            nodes = list(np.reshape(matrix, (1,-1))[0])
            for node in nodes:
                if np.isnan(node): nodes.remove(node)

            pos = 0
            for i in range(len(nodes)):
                if nodes[i] == v: return i
                pos += 1

        IM = np.zeros((get_m(self.VerticesMatrix), len(self.edges)))
        for i, item in enumerate(self.DictEdges.keys()):
            i1, i2 = self.DictEdges[item]
            IM[get_vert_poz(self.VerticesMatrix, i1)][i] = 1
            IM[get_vert_poz(self.VerticesMatrix, i2)][i] = 1
        return IM

    def pairs_for_break(self):
        pairs = list(self.edges)
        nums = [sorted(self.VerticesMatrix[0, :]),
                sorted(self.VerticesMatrix[:, -1]),
                sorted(self.VerticesMatrix[-1, :]),
                sorted(self.VerticesMatrix[:, 0])]
        for l in nums:
            for i in range(len(l)-1):
                p = (l[i], l[i+1])
                pairs.remove(p)
        return pairs

    def get_pairs_for_break(self):
        def CountFrequency(lst):
            freq = {i:0 for i in list(range(self.VerticesMatrix.size))}
            for item in lst: 
                freq[item] += 1
            return freq

        def check_pairs(pairs):
            usefull_pairs = []
            for case in pairs:
                Edges = [pair for pair in list(self.edges) if pair not in case]
                Edges_nums = []
                nums = set()
                for j in Edges:
                    Edges_nums += [j[0], j[1]]
                freq = CountFrequency(Edges_nums)

                for i in case:
                    nums.add(i[0])
                    nums.add(i[1])

                flag = False
                for k in nums:
                    if freq[k] == 1:
                        flag = True
                        
                if not flag: usefull_pairs.append(case)
            return usefull_pairs

        pairs = self.pairs_for_break
        
        # get all combinations for possible edges
        pairs_combs = []
        for i in range(1, len(pairs)+1):
            pairs_combs += list(it.combinations(pairs, i))
        return check_pairs(pairs_combs)

In [16]:
class SubGraph(Graph):
    def delete_edges(self, case):
        for pair in case:
            self.edges.remove(pair)
            pair_num = self.DictEdgesInv.pop(pair)
            self.DictEdges.pop(pair_num)
            self.DictResistance.pop(pair_num)
        
        # add NaN if node was deleted
        nums = []
        for pair in self.edges:
            a, b = pair
            if a not in nums: nums.append(a)
            if b not in nums: nums.append(b)
            
        for i in range(self.VerticesMatrix.shape[0]):
            for j in range(self.VerticesMatrix.shape[1]):
                if self.VerticesMatrix[i][j] not in nums: self.VerticesMatrix[i][j] = np.NaN

In [25]:
length_m, length_n = 3, 3

graph = Graph(length_m,length_n)

def get_vert_poz(matrix, v):
    nodes = list(np.reshape(matrix, (1,-1))[0])
    for node in nodes:
        if np.isnan(node): nodes.remove(node)

    pos = 0
    for i in range(len(nodes)):
        if nodes[i] == v: return i
        pos += 1

def calculate_case(graph):
    # (+1) for edge which represents connector
    num_edges = len(graph.edges) + 1
    Y = np.eye(num_edges)
    res = list(graph.DictResistance.values())
    for i in range(num_edges-1):
        Y[i][i] = res[i]
    Y[-1][-1] = 1/10
    J = np.zeros((num_edges, 1))
    J[-1] = 50

    fi_all = []

    # incidence matrix
    for i in graph.get_connections():
        A = graph.get_incidence_matrix()
        A = np.append(A, np.zeros((A.shape[0], 1)), axis=1)
        
        i1 = get_vert_poz(graph.VerticesMatrix, i[0])
        i2 = get_vert_poz(graph.VerticesMatrix, i[1])
        A[i2, -1] = 1
        A = np.delete(A, i1, axis=0)

        N = A@(Y@A.T)
        b = (-1)*(A@J)
    
        fi = np.linalg.inv(N)@b

        for i in range(A.shape[0]):
            if A[i, -1] == 1: fi = round(abs(fi[i][0]), 3)

        fi_all.append(fi)
    
    return fi_all

def get_data(graph):
    usefull_pairs_for_break = graph.get_pairs_for_break()
    n = len(graph.pairs_for_break)
    labels_order = {item:i for i, item in enumerate(graph.pairs_for_break)}
    
    # measurements and labels
    measurements = np.array([calculate_case(graph)])
    labels = np.array([[0]*n])

    for case in usefull_pairs_for_break:
        # GET LABEL
        label = [0]*n
        for item in case:
            label[labels_order[item]] = 1
        labels = np.append(labels, [label], axis=0)

        # GET MEASUREMENT
        gg = SubGraph(length_m,length_n)
        gg.delete_edges(case)
        measurements = np.append(measurements, [calculate_case(gg)], axis=0)

    return measurements, labels

measurements, labels = get_data(graph)

In [26]:
print(f'measurements shape: {measurements.shape}\n\
labels shape        {labels.shape}')

measurements shape: (12, 28)
labels shape        (12, 4)


In [49]:
# get the model
# CHANGE NUMBER OF NEURONES !!!
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(28, input_dim=n_inputs, kernel_initializer='he_uniform'))
    model.add(Dense(n_inputs*4, activation='relu'))
    model.add(Dense(n_inputs*8, activation='relu'))
    #model.add(Dense(35, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('> %.3f' % acc)
		results.append(acc)
	return results

In [50]:
# load dataset
X, y = measurements, labels
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))

> 1.000
> 0.667
> 1.000
> 0.500
> 0.500
> 0.333
> 0.000
> 0.000
> 1.000
> 0.500
> 0.333
> 0.667
> 0.500
> 0.500
> 1.000
Accuracy: 0.567 (0.321)
